In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing required libaries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# Importing the data


In [ ]:
df = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

df.head()

In [ ]:
df.shape

# Differentiating Categorical and Numerical Data.

In [ ]:
catFeatures = ['gender', 'hypertension', 'heart_disease','ever_married', 'work_type', 'Residence_type', 'smoking_status']
numFeatures = ['age', 'avg_glucose_level', 'bmi']

# Droping unecessary columns

In [ ]:
data = df.drop('id', axis=1)
data.head()

# Looking for unique values

In [ ]:
for catFeature in catFeatures:
    print(data[catFeature].unique())

# Checking for missing values.

In [ ]:
data.isna().sum()

# Plotting Countplots

In [ ]:
for catFeature in catFeatures:
    plt.figure(figsize=(10,5))
    sns.countplot(x = data[catFeature])
    plt.show()

In [ ]:
sns.pairplot(data)
plt.show()

We will treat unknown values as missing values.

In [ ]:
data['smoking_status'].replace('Unknown', np.nan, inplace=True)

# Missing Data.

In [ ]:
data.isna().sum()

# Filling the missing values.

In [ ]:
data['bmi'].fillna(data['bmi'].mean(), inplace=True)
data['smoking_status'].fillna(data['smoking_status'].mode()[0], inplace = True)

data.isna().sum()

# After filling the missing values countplot.

In [ ]:
sns.countplot(x = data['smoking_status'])

In [ ]:
plt.figure(figsize=(10,7))
sns.distplot(data["bmi"])
plt.show()

# Using Lable encoding

In [ ]:
le = LabelEncoder()

en_data = data[catFeatures].apply(le.fit_transform)

en_data.head()

# Feature Selection

In [ ]:
y = data['stroke']
X = pd.concat([en_data,data[numFeatures]], axis=1)

X.head()

# Unsampling the Dataset

In [ ]:
counter = Counter(y)
print(counter)

oversample = SMOTE()

X, y = oversample.fit_resample(X,y)

counter = Counter(y)
print(counter)

# Splitting test and train set.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

# Applying Standardization.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Training model using RandomForestClassifier.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators=50)
model_rf.fit(X_train, y_train)
y_pred = model_rf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("The Training Score of RandomForestClassifier is: {:.3f}%".format(model_rf.score(X_train, y_train)*100))
print("\n----------------------------------------------------------------------\n")
print("The Classification report: \n{}\n".format(classification_report(y_test, y_pred)))

#Visualize confusion matrix
plt.figure(figsize = (10, 8))
sns.heatmap(confusion_matrix(y_test, y_pred), cmap = 'Blues', annot = True, fmt = 'd', linewidths = 5, cbar = False, annot_kws = {'fontsize': 15},
           yticklabels = ['No stroke', 'Stroke'], xticklabels = ['Predicted no stroke', 'Predicted stroke'])
plt.yticks(rotation = 0)
plt.show()

print("\n----------------------------------------------------------------------\n") 
print("The Accuracy Score of RandomForestClassifier is: {:.3f}%".format(accuracy_score(y_test, y_pred)*100))